# Project 1

### Importing Data

In [148]:
import pandas as pd


In [149]:
df = pd.read_excel("2018_Statistical_Annex_Table_1.xlsx",index_col = False)


### Initial Interpretations

In [150]:
#we use the head function to get a quick sense of the structure of the data
df.head(20)

,Table 1. Human Development Index and its components,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,NaN,NaN,NaN,NaN,SDG 3,NaN,SDG 4.3,NaN,SDG 4.6,NaN,SDG 8.5,NaN,NaN,NaN,NaN
1,NaN,NaN,Human Development Index (HDI),NaN,Life expectancy at birth,NaN,Expected years of schooling,NaN,Mean years of schooling,NaN,Gross national income (GNI) per capita,NaN,GNI per capita rank minus HDI rank,NaN,HDI rank
2,HDI rank,Country,Value,NaN,(years),NaN,(years),NaN,(years),NaN,(2011 PPP $),NaN,NaN,NaN,NaN
3,NaN,NaN,2017,NaN,2017,NaN,2017,a,2017,a,2017,NaN,2017,NaN,2016
4,NaN,VERY HIGH HUMAN DEVELOPMENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,Norway,0.952522,NaN,82.328,NaN,17.8521,NaN,12.5668,NaN,68012.5,NaN,5,NaN,1
6,2,Switzerland,0.943998,NaN,83.473,NaN,16.2088,NaN,13.408,NaN,57625.1,NaN,8,NaN,2
7,3,Australia,0.938631,NaN,83.068,NaN,22.9213,b,12.855,NaN,43560.1,NaN,18,NaN,3
8,4,Ireland,0.93841,NaN,81.643,NaN,19.6137,b,12.5263,c,53754.2,NaN,8,NaN,4
9,5,Germany,0.936043,NaN,81.178,NaN,16.956,NaN,14.082,NaN,46135.8,NaN,13,NaN,4


We see that the data is tabulated with some custom nested headers, the field names are spread across rows 3 and 4. In order to make the data more compatible the pandas dataframe structure we will remove the extraneous headings and have the data entries properly aligned. 

In [151]:
data = df[5:]

In [152]:
data.head(10)

,Table 1. Human Development Index and its components,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
5,1,Norway,0.952522,NaN,82.328,NaN,17.8521,NaN,12.5668,NaN,68012.5,NaN,5,NaN,1
6,2,Switzerland,0.943998,NaN,83.473,NaN,16.2088,NaN,13.408,NaN,57625.1,NaN,8,NaN,2
7,3,Australia,0.938631,NaN,83.068,NaN,22.9213,b,12.855,NaN,43560.1,NaN,18,NaN,3
8,4,Ireland,0.93841,NaN,81.643,NaN,19.6137,b,12.5263,c,53754.2,NaN,8,NaN,4
9,5,Germany,0.936043,NaN,81.178,NaN,16.956,NaN,14.082,NaN,46135.8,NaN,13,NaN,4
10,6,Iceland,0.934879,NaN,82.912,NaN,19.3479,b,12.3638,c,45810.2,NaN,13,NaN,6
11,7,"Hong Kong, China (SAR)",0.932583,NaN,84.097,NaN,16.3257,NaN,12.0381,NaN,58419.7,NaN,2,NaN,8
12,7,Sweden,0.932805,NaN,82.625,NaN,17.6346,NaN,12.4261,NaN,47765.7,NaN,9,NaN,7
13,9,Singapore,0.932042,NaN,83.218,NaN,16.2,d,11.4747,NaN,82503.1,e,-6,NaN,8
14,10,Netherlands,0.930639,NaN,82.005,NaN,18.0448,NaN,12.19,NaN,47899.8,NaN,5,NaN,10


In [153]:
headers = df.iloc[1].values.tolist()

In [154]:
headers

[nan,
 nan,
 'Human Development Index (HDI) ',
 nan,
 'Life expectancy at birth',
 nan,
 'Expected years of schooling ',
 nan,
 'Mean years of schooling',
 nan,
 'Gross national income (GNI) per capita',
 nan,
 'GNI per capita rank minus HDI rank',
 nan,
 'HDI rank']

In [155]:
headers[0] = "HDI Rank"
headers[1] = "Country"

In [156]:
data.columns = headers

In [157]:
data.head()

,HDI Rank,Country,Human Development Index (HDI),nan,Life expectancy at birth,nan,Expected years of schooling,nan,Mean years of schooling,nan,Gross national income (GNI) per capita,nan,GNI per capita rank minus HDI rank,nan,HDI rank
5,1,Norway,0.952522,NaN,82.328,NaN,17.8521,NaN,12.5668,NaN,68012.5,NaN,5,NaN,1
6,2,Switzerland,0.943998,NaN,83.473,NaN,16.2088,NaN,13.408,NaN,57625.1,NaN,8,NaN,2
7,3,Australia,0.938631,NaN,83.068,NaN,22.9213,b,12.855,NaN,43560.1,NaN,18,NaN,3
8,4,Ireland,0.93841,NaN,81.643,NaN,19.6137,b,12.5263,c,53754.2,NaN,8,NaN,4
9,5,Germany,0.936043,NaN,81.178,NaN,16.956,NaN,14.082,NaN,46135.8,NaN,13,NaN,4


In [158]:
data = data.loc[:, data.columns.notnull()]
data.head()

,HDI Rank,Country,Human Development Index (HDI),Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita,GNI per capita rank minus HDI rank,HDI rank
5,1,Norway,0.952522,82.328,17.8521,12.5668,68012.5,5,1
6,2,Switzerland,0.943998,83.473,16.2088,13.408,57625.1,8,2
7,3,Australia,0.938631,83.068,22.9213,12.855,43560.1,18,3
8,4,Ireland,0.93841,81.643,19.6137,12.5263,53754.2,8,4
9,5,Germany,0.936043,81.178,16.956,14.082,46135.8,13,4


Our dataset looks clean! However we can still count the null values.

In [159]:
data.isnull().sum().sum()

424

In [160]:
data.tail(69)

,HDI Rank,Country,Human Development Index (HDI),Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita,GNI per capita rank minus HDI rank,HDI rank
196,189,Niger,0.353931,60.422,5.36851,1.95133,905.868,-2,188
197,NaN,OTHER COUNTRIES OR TERRITORIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,..,Korea (Democratic People's Rep. of),..,71.887,12.0002,..,..,..,..
199,..,Monaco,..,..,..,..,..,..,..
200,..,Nauru,..,..,10.3143,..,18573,..,..
201,..,San Marino,..,..,15.1112,..,..,..,..
202,..,Somalia,..,56.714,..,..,..,..,..
203,..,Tuvalu,..,..,..,..,5887.72,..,..
204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
205,NaN,Human development groups,NaN,NaN,NaN,NaN,NaN,NaN,NaN


It appears our data is still full of null values and the end of our dataset is the issue. We can remove these problematic rows by keeping only rows with numeric values for 'HDI rank'. 

In [161]:
data = data[pd.to_numeric(data['HDI rank'], errors='coerce').notnull()]
data.isnull().sum().sum() # count the null values again

0

In [162]:
data.head(69)

,HDI Rank,Country,Human Development Index (HDI),Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita,GNI per capita rank minus HDI rank,HDI rank
5,1,Norway,0.952522,82.328,17.8521,12.5668,68012.5,5,1
6,2,Switzerland,0.943998,83.473,16.2088,13.408,57625.1,8,2
7,3,Australia,0.938631,83.068,22.9213,12.855,43560.1,18,3
8,4,Ireland,0.93841,81.643,19.6137,12.5263,53754.2,8,4
9,5,Germany,0.936043,81.178,16.956,14.082,46135.8,13,4
10,6,Iceland,0.934879,82.912,19.3479,12.3638,45810.2,13,6
11,7,"Hong Kong, China (SAR)",0.932583,84.097,16.3257,12.0381,58419.7,2,8
12,7,Sweden,0.932805,82.625,17.6346,12.4261,47765.7,9,7
13,9,Singapore,0.932042,83.218,16.2,11.4747,82503.1,-6,8
14,10,Netherlands,0.930639,82.005,18.0448,12.19,47899.8,5,10


In [164]:
data.to_csv("out.csv")